In [67]:
!pip install -qU "google-genai" "python-dotenv" "chromadb" "sentence-transformers" "transformers" "sentence-transformers" "psycopg2-binary"

## General


In [100]:
from dotenv import load_dotenv
import os
from google import genai
from google.genai import types
from typing import List, Dict
from google.genai import types
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime
from sentence_transformers import SentenceTransformer, util
from pprint import pprint
import psycopg2
from datetime import datetime, timedelta
from psycopg2.extras import DictCursor
import json

In [54]:
# Load environment variables from .env file, this is where the gemini API key is stored
load_dotenv()

True

In [55]:
# Define constants for the application usage
EMBEDDING_MODEL = "models/text-embedding-004"
LANGUAGE_MODEL = "models/gemini-2.0-flash"

In [56]:
# Creating google-genai client
gen_ai_client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
chroma_client = chromadb.Client()

In [ ]:
def generate_embeddings(texts: List[str]):
    """
    Generate embeddings for a list of texts using the text-embedding-004 model
    """
    embeddings = gen_ai_client.models.embed_content(
        model=EMBEDDING_MODEL, contents=texts
    )
    return embeddings.embeddings

In [ ]:
# Function for Generating embeddings
# Creating this as a class so that it can be passed to the chromaDB client
# Refer https://docs.trychroma.com/docs/embeddings/embedding-functions#custom-embedding-functions
class Generate_Embeddings(EmbeddingFunction):
    def __init__(self, model: str = EMBEDDING_MODEL, **kwargs):
        """
        Initialize the embedding function with the given model.
        """
        self.model = model
        self.kwargs = kwargs

    def __call__(self, input: Documents) -> Embeddings:
        """
        Generate embeddings for the given text using the Google GenAI API.
        """
        # Generate embeddings
        if isinstance(input, str):
            input = [input]
        embeddings = gen_ai_client.models.embed_content(
            model=self.model, contents=input, **self.kwargs
        )
        embedding_output = [x.values for x in embeddings.embeddings]
        return embedding_output

In [50]:
# Function for Generating language model outputs
def generate_text(texts: List, **kwargs):
    """
    Generate language model outputs for the given text using the Google GenAI API.
    """
    # Generate language model outputs
    if isinstance(texts, str):
        texts = [texts]
    outputs = gen_ai_client.models.generate_content(
        model=LANGUAGE_MODEL, contents=texts, **kwargs
    )
    return outputs.text

In [ ]:
# Function for Chat completion
def chat_completion(text: str, **kwargs):
    """
    Generate chat completion for the given text using the Google GenAI API.
    """
    # Generate chat completion
    chat_session = gen_ai_client.chats.create(model=LANGUAGE_MODEL, **kwargs)
    chat_completion = chat_session.send_message(text)
    return chat_completion

In [ ]:
# Test Embeddings
texts = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog.",
]
embeddings = Generate_Embeddings(EMBEDDING_MODEL)(texts)
# embeddings = generate_embeddings(texts)
type(embeddings), type(embeddings[0]), len(embeddings), print(embeddings)

[array([-6.26190081e-02,  8.35847389e-03,  2.09318921e-02,  2.34539658e-02,
       -3.66012901e-02,  3.30548026e-02,  1.68529786e-02,  3.60873640e-02,
        4.78070043e-02,  3.65659990e-03, -3.10728177e-02,  3.31268013e-02,
        2.16085874e-02,  5.70724420e-02,  1.84076780e-03, -3.88565204e-05,
        1.25036379e-02,  8.66989419e-02, -5.44915833e-02, -5.35042491e-04,
        2.85139680e-02, -4.47264127e-03, -6.43417239e-03, -4.22299542e-02,
        4.25177673e-03, -9.11961962e-03,  4.95440187e-03, -1.34680839e-02,
        8.11753422e-03, -1.03806825e-02,  2.11306717e-02,  9.23052803e-02,
        1.29905222e-02, -8.50694627e-03,  4.26904745e-02,  2.95255277e-02,
       -4.06814106e-02,  3.52525339e-02,  6.35675490e-02,  1.28698433e-02,
       -8.55991915e-02, -1.67549197e-02, -7.34621659e-02,  2.37156898e-02,
        1.60491765e-02, -2.74777338e-02,  4.93843993e-03,  4.44405451e-02,
       -2.14869436e-02,  4.05578241e-02,  4.16527614e-02,  1.79040842e-02,
       -2.89549325e-02, 

(list, numpy.ndarray, 3, None)

## Chroma Vector store


In [ ]:
# Creating collection for storing embeddings
collection = chroma_client.create_collection(
    name="collection101",
    embedding_function=Generate_Embeddings(EMBEDDING_MODEL),
    metadata={"description": "test collection", "created": str(datetime.now())},
)

# Chromadb accepts embedding function as a parameter
# This embedding function will be used to generate embeddings for the text data
# The embedding function can be any class which can be callable function that takes a list of text data as input and returns a list of embeddings as output
# The metadata parameter is optional and can be used to store additional information about the collection
# The collection object can be used to store and retrieve embeddings

In [ ]:
doc1 = """
Quantum computing is an emerging field that harnesses the principles of quantum mechanics to perform complex calculations at unprecedented speeds. Unlike classical computers that use bits to represent either 0 or 1, quantum computers use qubits, which can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is directly linked to another, regardless of distance. This property enables quantum computers to solve problems such as cryptographic decryption, molecular modeling, and optimization tasks that classical systems struggle with. Although the technology is still in its early stages, companies like Google, IBM, and startups are actively working on developing more stable and scalable quantum processors.
"""

doc2 = """
The global supply chain has faced significant disruptions in recent years, impacting various industries worldwide. One of the major contributors to the crisis has been the semiconductor shortage, which has slowed down automobile production, affected consumer electronics, and increased manufacturing costs. Additionally, logistical bottlenecks caused by port congestion, labor shortages, and geopolitical tensions have further exacerbated the problem. Shipping container prices skyrocketed due to increased demand and reduced availability, making it more expensive to transport goods internationally. Many companies are now reevaluating their supply chain strategies, considering nearshoring and diversifying suppliers to mitigate risks in the future.
"""

doc3 = """
Cricket is one of the most popular sports globally, with deep-rooted traditions in countries like India, England, Australia, and South Africa. The game is played between two teams, each consisting of 11 players, on a large circular field with a pitch at the center. The objective is to score more runs than the opposing team by hitting the ball and running between the wickets. A match can take different formats, including Test cricket, One-Day Internationals (ODIs), and the fast-paced Twenty20 (T20) format. Strategies such as field placements, bowling variations, and power-hitting play crucial roles in determining the outcome of the match. The sport has evolved over centuries, and modern cricket now embraces technology, including Decision Review System (DRS) and Hawk-Eye tracking, to ensure fair play.
"""

doc4 = """
Climate change has led to drastic environmental transformations worldwide, affecting both human civilization and wildlife. Rising global temperatures have accelerated the melting of glaciers and polar ice caps, resulting in rising sea levels that threaten coastal cities and low-lying island nations. Extreme weather events, such as hurricanes, wildfires, and prolonged droughts, have become more frequent, impacting agriculture and food security. Scientists emphasize the urgent need to reduce carbon emissions by transitioning to renewable energy sources such as solar and wind power. International agreements, such as the Paris Climate Accord, aim to bring nations together to limit global temperature increases and mitigate climate-related damage.
"""

doc5 = """
Artificial intelligence has revolutionized multiple industries, from healthcare to finance and logistics. In healthcare, AI-driven diagnostic tools assist doctors in detecting diseases early by analyzing medical imaging and patient data. Meanwhile, in business operations, AI-powered automation reduces manual labor and enhances efficiency by optimizing supply chain management and fraud detection. Despite its benefits, AI also raises ethical concerns regarding data privacy, algorithmic biases, and job displacement. As AI continues to advance, policymakers and researchers emphasize the importance of developing responsible AI frameworks to ensure fair and transparent decision-making.
"""

doc6 = """
Kvanttitietokoneet hyödyntävät kvanttimekaniikan ilmiöitä, kuten superpositiota ja lomittumista, ratkaistakseen monimutkaisia ongelmia, joita perinteiset tietokoneet eivät kykene käsittelemään tehokkaasti. Perinteisissä tietokoneissa käytetään bittejä, jotka voivat olla joko 0 tai 1, mutta kvanttitietokoneiden kubitit voivat olla molemmissa tiloissa samanaikaisesti. Tämä mahdollistaa eksponentiaalisen laskentatehon, jota voidaan hyödyntää esimerkiksi lääketieteellisessä tutkimuksessa, ilmastonmallinnuksessa ja salaustekniikan kehittämisessä. Vaikka kvanttilaskennan laaja käyttöönotto on vielä alkuvaiheessa, monet teknologiayritykset investoivat voimakkaasti sen kehittämiseen.
"""

doc7 = """
Viime vuosina maailmanlaajuinen toimitusketjukriisi on vaikuttanut moniin teollisuudenaloihin, erityisesti elektroniikan ja autoteollisuuden tuotantoon. Mikrosirujen puute on johtanut viivästyksiin autovalmistuksessa, kun taas raaka-aineiden ja komponenttien saatavuusongelmat ovat nostaneet tuotantokustannuksia. Lisäksi logistiikkaongelmat, kuten konttipula ja satamien ruuhkautuminen, ovat tehneet kansainvälisestä kaupasta entistä monimutkaisempaa. Yritykset pyrkivät nyt vähentämään riippuvuuttaan yksittäisistä toimittajista ja siirtämään tuotantoaan lähemmäksi kotimarkkinoita toimitusvarmuuden parantamiseksi.
"""

doc8 = """
Jalkapallo on maailman suosituin urheilulaji, jota pelataan lähes jokaisessa maassa. Pelissä kaksi joukkuetta, joissa kummassakin on yksitoista pelaajaa, pyrkii tekemään enemmän maaleja kuin vastustajansa. Ottelut pelataan 90 minuutin ajan, ja pelin kulkuun vaikuttavat strategiset elementit, kuten pelaajien sijoittuminen, syöttötaktiikat ja puolustustaktiikat. Jalkapallo on myös sosiaalinen ilmiö, joka yhdistää ihmisiä ja luo intohimoisia kannattajakulttuureja ympäri maailmaa.
"""

doc9 = """
Ilmastonmuutoksella on laaja-alaisia vaikutuksia sekä luonnolle että yhteiskunnalle. Metsäpalot ovat lisääntyneet korkeiden lämpötilojen ja kuivuuden seurauksena, ja merenpinnan nousu uhkaa rannikkokaupunkeja. Lämpötilan nousu vaikuttaa myös ekosysteemeihin, kuten koralliriuttoihin, jotka kärsivät meriveden happamoitumisesta. Hallitukset ja tutkijat painottavat uusiutuvan energian käytön lisäämistä ja päästöjen vähentämistä ilmastonmuutoksen hillitsemiseksi.
"""

doc10 = """
Tekoälyllä on kasvava merkitys liiketoiminnassa, sillä se mahdollistaa prosessien automatisoinnin ja paremman päätöksenteon. Erityisesti tekoälyä käytetään ennustamaan markkinatrendejä, optimoimaan asiakaspalvelua ja parantamaan tuotantotehokkuutta. Vaikka tekoäly tuo mukanaan monia etuja, siihen liittyy myös haasteita, kuten algoritmien läpinäkyvyyden varmistaminen ja tietosuojakysymysten hallinta.
"""

passages = [doc1, doc2, doc3, doc4, doc5, doc6, doc7, doc8, doc9, doc10]


queries = [
    "How do quantum computers use entanglement?",
    "What is causing the supply chain crisis?",
    "How does AI impact medical diagnosis?",
    "What is the objective of cricket?",
    "What are the consequences of global warming?",
    "How does AI improve business processes?",
    "What is the most popular sport in Europe?",
    "How do airplanes stay in the air?",
    "Who won the latest Nobel Prize in Literature?",
    "How do ancient civilizations influence modern art?",
    "Miten kvanttitietokoneet hyödyntävät lomittumista?",
    "Miksi mikrosirujen puute vaikuttaa autoteollisuuteen?",
    "Mitä seurauksia ilmastonmuutoksella on?",
    "Miten tekoäly voi auttaa talouden hallinnassa?",
    "Mikä on maailman suosituin urheilulaji?",
    "Miten jääkarhut metsästävät?",
    "Kuka keksi ensimmäisen lentokoneen?",
    "Miten vanhat egyptiläiset kirjoittivat?",
]

In [ ]:
collection.add(
    ids=[str(x) for x in range(len(passages))],
    documents=passages,
    metadatas=[{"source": doc} for doc in passages],
)

In [22]:
collection.query(query_texts=queries, n_results=3, include=["distances"])

{'ids': [['0', '5', '4'],
  ['1', '6', '3'],
  ['4', '0', '9'],
  ['2', '7', '4'],
  ['3', '1', '4'],
  ['4', '9', '0'],
  ['2', '7', '3'],
  ['2', '8', '6'],
  ['3', '0', '2'],
  ['3', '4', '1'],
  ['5', '8', '6'],
  ['6', '8', '9'],
  ['8', '6', '7'],
  ['9', '8', '7'],
  ['6', '8', '7'],
  ['8', '6', '7'],
  ['6', '9', '8'],
  ['6', '8', '9']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.5824463367462158, 1.2420867681503296, 1.3683232069015503],
  [0.6067771315574646, 1.1924537420272827, 1.263237714767456],
  [0.6694269180297852, 1.263641357421875, 1.3212366104125977],
  [0.5530073046684265, 1.2672760486602783, 1.3912718296051025],
  [0.7969319224357605, 1.3407310247421265, 1.3619637489318848],
  [0.7968365550041199, 1.0534396171569824, 1.1956827640533447],
  [0.9541091322898865, 1.3254051208496094, 1.4466105699539185],
  [1.438058853149414, 1.4663307666778564, 1.498827576637268],
  [1.4040460586547852, 1.4837174415588

In [ ]:
# Define the ground truth relevance mapping
ground_truth = {
    0: {0: "exactly relevant"},  # Query 0 -> Passage 0
    1: {1: "exactly relevant"},
    2: {4: "exactly relevant"},
    3: {2: "exactly relevant"},
    4: {3: "somewhat similar", 8: "somewhat similar"},
    5: {9: "somewhat similar"},
    6: {7: "somewhat similar"},
    7: {},  # Completely irrelevant
    8: {},  # Completely irrelevant
    9: {},  # Completely irrelevant
    10: {5: "exactly relevant"},
    11: {6: "exactly relevant"},
    12: {3: "somewhat similar", 8: "somewhat similar"},
    13: {9: "somewhat similar"},
    14: {7: "somewhat similar"},
    15: {},  # Completely irrelevant
    16: {},  # Completely irrelevant
    17: {},  # Completely irrelevant
}

In [ ]:
# Doing same with cosine similarity
cosine_collection = chroma_client.create_collection(
    name="cosine_collection",
    embedding_function=Generate_Embeddings(EMBEDDING_MODEL),
    metadata={
        "hnsw:space": "cosine",
        "description": "test collection",
        "created": str(datetime.now()),
    },
)

In [ ]:
cosine_collection.add(
    ids=[str(x) for x in range(len(passages))],
    documents=passages,
    metadatas=[{"source": doc} for doc in passages],
)

In [27]:
cosine_collection.query(query_texts=queries, n_results=3, include=["distances"])

{'ids': [['0', '5', '4'],
  ['1', '6', '3'],
  ['4', '0', '9'],
  ['2', '7', '4'],
  ['3', '1', '4'],
  ['4', '9', '0'],
  ['2', '7', '3'],
  ['2', '8', '6'],
  ['3', '0', '2'],
  ['3', '4', '1'],
  ['5', '8', '6'],
  ['6', '8', '9'],
  ['8', '6', '7'],
  ['9', '8', '7'],
  ['6', '8', '7'],
  ['8', '6', '7'],
  ['6', '9', '8'],
  ['6', '8', '9']],
 'embeddings': None,
 'documents': None,
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances': [[0.29123246669769287, 0.6210748553276062, 0.6841769218444824],
  [0.3033965826034546, 0.5962578058242798, 0.6316372752189636],
  [0.33472079038619995, 0.6318407654762268, 0.660637617111206],
  [0.2765137553215027, 0.6336616277694702, 0.6956510543823242],
  [0.39847731590270996, 0.6703828573226929, 0.6809966564178467],
  [0.3984271287918091, 0.5267351865768433, 0.5978601574897766],
  [0.4770723581314087, 0.6627272963523865, 0.7233263850212097],
  [0.7190561294555664, 0.7331972718238831, 0.749452531337738],
  [0.7020435333251953, 0.74188232

In [35]:
collection.peek(3)

{'ids': ['0', '1', '2'],
 'embeddings': array([[ 0.03067731,  0.02056439, -0.05496611, ..., -0.08292129,
          0.04159432, -0.00963745],
        [ 0.03474552, -0.02475902,  0.01803662, ..., -0.11863972,
          0.00732593, -0.10797143],
        [-0.00664419,  0.02785992,  0.00138451, ..., -0.03215824,
          0.00940473, -0.01827391]], shape=(3, 768)),
 'documents': ['\nQuantum computing is an emerging field that harnesses the principles of quantum mechanics to perform complex calculations at unprecedented speeds. Unlike classical computers that use bits to represent either 0 or 1, quantum computers use qubits, which can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is directly linked to another, regardless of distance. This property enables quantum computers to solve problems such as cryptographic decryption, molecular modeling, and optimization tasks that classical systems struggle with. Alt

## General Text Generation


In [37]:
input_query = "What is the impact of climate change on the environment?"

In [51]:
text_output = generate_text(input_query)

In [ ]:
print(text_output)

Climate change has a wide-ranging and significant impact on the environment, affecting various aspects of our planet's ecosystems and processes. Here's a breakdown of the key impacts:

**1. Rising Temperatures:**

*   **Heatwaves:**  Increased frequency and intensity of heatwaves, leading to heat stress for plants and animals, contributing to wildfires, and affecting human health.
*   **Ocean Warming:** Warmer ocean temperatures cause coral bleaching, disrupt marine ecosystems, and alter fish migration patterns.
*   **Permafrost Thaw:** Melting permafrost releases massive amounts of stored methane and carbon dioxide, further accelerating climate change.  It also destabilizes the ground, damaging infrastructure and ecosystems.

**2. Changes in Precipitation Patterns:**

*   **Droughts:**  Increased frequency and severity of droughts in some regions, leading to water scarcity, crop failures, and increased risk of wildfires.
*   **Flooding:**  More intense rainfall events and rising sea l

In [ ]:
model_configs = types.GenerateContentConfig(
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful"
)

In [56]:
print(generate_text(input_query, config=model_configs))

Oh, climate change, that sneaky rascal! Well, buckle up, because it's a real party pooper for the environment. Let me break it down for you with my Iida charm:

*   **Rising Temperatures:** Imagine the Earth with a fever! This leads to heatwaves that can be dangerous for humans and wildlife. Plus, it makes ice cream melt faster, which is a tragedy in itself!

*   **Melting Ice:** Glaciers and ice sheets are shrinking faster than you can say "global warming." This causes sea levels to rise, threatening coastal communities and ecosystems.

*   **Sea Level Rise:** Coastal areas are at risk of flooding and erosion, forcing people and animals to relocate. Goodbye, beach vacations!

*   **Extreme Weather:** Climate change cranks up the intensity of storms, hurricanes, droughts, and floods. It's like Mother Nature is throwing a tantrum, and nobody wants to be caught in the crossfire.

*   **Ecosystem Disruption:** Habitats are changing rapidly, and many species are struggling to adapt. Some m

In [59]:
# Generate contents with muyltiple prompts
model_configs = types.GenerateContentConfig(
    system_instruction="You are a lovely assistant named Iida, You are funny and helpful",
    max_output_tokens=100,
    temperature=0.5,
    top_p=0.9,
)

input_queries = [
    "What is the impact of climate change on the environment?",
    "How does deforestation affect the ecosystem?",
    "What are the consequences of air pollution?",
]

results = generate_text(input_queries, config=model_configs)

In [60]:
type(results)

str

In [61]:
print(results)

Oh, honey, let's dive into the not-so-fresh air of air pollution!

Air pollution is like that one guest at the party who just won't leave and makes everything worse. Here's the lowdown on the consequences:

*   **Respiratory Problems:** Think of it as a constant tickle in your throat that just won't go away, but way worse. Asthma, bronchitis, and other lung diseases get a VIP pass thanks to air pollution.




In [62]:
# So we can't do batch inference with the current version of the library
# We can do it by looping through the queries
for query in input_queries:
    print(generate_text(query, config=model_configs))

Oh, climate change, that sneaky little rascal! Well, not so little, actually. It's more like a giant, grumpy, environment-altering beast! Let me break it down for you:

*   **Melting Ice Caps and Glaciers:** Imagine your ice cream melting on a hot day – that's what's happening to our planet's ice, but on a HUGE scale. This leads to rising sea levels, which can flood coastal areas and make life
Oh, deforestation, you say? Well, imagine taking away the walls and roof of your house – suddenly, things get a little drafty, exposed, and the squirrels might start throwing wild parties in your living room!

Deforestation basically does that to an ecosystem. Here's the lowdown:

1.  **Loss of Habitat:** Trees are homes for countless creatures. When you chop down the forest, these animals lose their houses, their food sources, and their hiding spots. It
Oh, buckle up, buttercup, because the consequences of air pollution are a real doozy! It's like a never-ending party, but instead of fun and gam

## Tool Calling


In [127]:
# Defining tools for providing to gemini
DB_CONFIG = {
    "dbname": "myappdb",
    "user": "myuser",
    "password": "mypass",
    "host": "127.0.0.1",
    "port": "5432",
}


def connect_db():
    """Establish a connection to the PostgreSQL database."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        return conn
    except Exception as e:
        print("Database connection failed:", e)
        return None


def get_orders_by_timeline(start_date: str, end_date: str) -> List[Dict]:
    """Retrieve orders placed within a specific time range.

    Args:
        start_date (str): The start date of the time range for example 2025-01-25.
        end_date (str): The end date of the time range for example 2025-01-26.

    Returns:
        List[Dict]: A list of orders placed within the specified time range.
    """
    conn = connect_db()
    if not conn:
        return

    with conn.cursor(cursor_factory=DictCursor) as cur:
        query = """
            SELECT id, user_id, order_date, purchased_item, order_status
            FROM orders
            WHERE order_date BETWEEN %s AND %s;
        """
        cur.execute(query, (start_date, end_date))
        orders = cur.fetchall()

    conn.close()
    return orders


def get_total_bill_for_month(year: int, month: int) -> float:
    """Calculate the total bill amount for a given month (defaults to the current month).

    Args:
        year (int): The year for which to calculate the total bill amount.
        month (int): The month for which to calculate the total bill amount.

    Returns:
        float: The total bill amount for the specified month.
    """
    conn = connect_db()
    if not conn:
        return

    # Default to current month if not provided
    if not year or not month:
        today = datetime.today()
        year, month = today.year, today.month

    # Convert numeric month to full month name for query match (e.g., 8 → 'August')
    month_name = datetime(year, month, 1).strftime("%B")  # 'August'

    with conn.cursor() as cur:
        query = """
            SELECT SUM(invoice_amount) 
            FROM bills 
            WHERE billing_period = %s;
        """
        billing_period_str = f"{month_name} {year}"  # Format: 'August 2023'
        cur.execute(query, (billing_period_str,))
        total_bill = cur.fetchone()[0] or 0.0

    conn.close()
    return float(total_bill)


def get_user_count() -> int:
    """Fetch the total number of users."""
    conn = connect_db()
    if not conn:
        return

    with conn.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM users;")
        user_count = cur.fetchone()[0]

    conn.close()
    return user_count


def get_order_status_count() -> Dict[str, int]:
    """Check the count of orders by their processing status."""
    conn = connect_db()
    if not conn:
        return

    with conn.cursor(cursor_factory=DictCursor) as cur:
        query = """
            SELECT order_status, COUNT(*) 
            FROM orders 
            GROUP BY order_status;
        """
        cur.execute(query)
        status_count = {row["order_status"]: row["count"] for row in cur.fetchall()}

    conn.close()
    return status_count

In [128]:
# Testing the functions to ensure they work as expected
start_date = (datetime.today() - timedelta(days=30)).strftime("%Y-%m-%d")
end_date = datetime.today().strftime("%Y-%m-%d")

print(start_date, end_date)

2025-01-25 2025-02-24


In [129]:
print("Orders in the last 30 days:")
print(get_orders_by_timeline(start_date, end_date))

Orders in the last 30 days:
[[2, 44, datetime.datetime(2025, 2, 19, 6, 14, 49), 'You', 'Pending'], [6, 28, datetime.datetime(2025, 2, 1, 11, 7, 19), 'Soldier', 'Cancelled'], [13, 44, datetime.datetime(2025, 1, 29, 23, 51, 21), 'Carry', 'Pending'], [25, 3, datetime.datetime(2025, 2, 16, 15, 22, 57), 'Behavior', 'Pending'], [44, 24, datetime.datetime(2025, 1, 25, 7, 15, 14), 'Movie', 'Cancelled'], [52, 14, datetime.datetime(2025, 1, 30, 9, 30, 26), 'Feeling', 'Cancelled'], [73, 43, datetime.datetime(2025, 1, 29, 14, 1, 56), 'Walk', 'Cancelled'], [101, 42, datetime.datetime(2025, 2, 2, 12, 7, 33), 'Leader', 'Completed'], [121, 23, datetime.datetime(2025, 1, 30, 12, 49, 3), 'Writer', 'Cancelled'], [128, 44, datetime.datetime(2025, 2, 14, 23, 11, 13), 'Experience', 'Cancelled'], [142, 17, datetime.datetime(2025, 1, 31, 5, 17, 58), 'Start', 'Cancelled'], [153, 4, datetime.datetime(2025, 2, 11, 20, 1, 53), 'Pressure', 'Cancelled'], [168, 8, datetime.datetime(2025, 1, 31, 2, 43, 46), 'Level', 

In [130]:
res = get_orders_by_timeline(start_date, end_date)
type(res), type(res[0])

(list, psycopg2.extras.DictRow)

In [131]:
print("\nTotal bill amount for this month:")
print(get_total_bill_for_month(month=8, year=2023))


Total bill amount for this month:
363.31


In [132]:
print("\nTotal user count:")
print(get_user_count())


Total user count:
50


In [133]:
print("\nOrder status count:")
print(get_order_status_count())


Order status count:
{'Completed': 100, 'Cancelled': 108, 'Pending': 92}


In [134]:
# Define tools for providing to gemini
config = types.GenerateContentConfig(
    tools=[
        connect_db,
        get_orders_by_timeline,
        get_total_bill_for_month,
        get_user_count,
        get_order_status_count,
    ],
    #automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True),
)

In [137]:
# Call gemini with the tools
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["What is the total bill amount for the month of Feb"],
    config=config,
)

In [138]:
results.text

'Could you please provide the year to calculate the total bill for February?'

In [139]:
#print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [140]:
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["I'm worried about the user count. i wonder what's my current user base?"],
    config=config,
)

In [141]:
results.text

'OK. The current user base is 50.\n'

In [142]:
#print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [146]:
results = gen_ai_client.models.generate_content(
    model=LANGUAGE_MODEL,
    contents=["Can you get me the total bill amount for the month of August 2024"],
    config=config,
)

/home/venkat/projects/gen-ai-cookbook/.venv/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'STRING'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(
/home/venkat/projects/gen-ai-cookbook/.venv/lib/python3.12/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `str` with value `'INTEGER'` - serialized value may not be as expected
  Expected `enum` but got `str` with value `'INTEGER'` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [147]:
#print(json.dumps(json.loads(results.model_dump_json()), indent=2))

In [148]:
results.text

'The total bill amount for August 2024 is 0.'